# Upload Structure into AiiDA

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.data.structure import StructureData

import ase.io
import ipywidgets as widgets
from IPython.display import display, clear_output
from fileupload import FileUploadWidget
from tempfile import NamedTemporaryFile

## Step 1: Upload file

In [ ]:
def on_file_upload(c):
    clear_output()
    global traj
    tmp = NamedTemporaryFile(suffix=file_upload.filename)
    f = open(tmp.name, "w")
    f.write(file_upload.data)
    f.close()
    traj = ase.io.read(tmp.name, index=":")
    tmp.close()
    #update_view()
    print("Found %d structures:"%len(traj))
    print(traj)
    
#TODO: FileUploadWidget doesn't fire event when same file is uploaded twice
file_upload = FileUploadWidget("Upload Structure")
file_upload.observe(on_file_upload, names='data')

display(file_upload)

## Step 2: Store in AiiDA Database

In [ ]:
def on_click_store(b):
    clear_output()
    global traj
    for atoms in traj:
        s = StructureData(ase=atoms)
        s.description = inp_descr.value
        s.store()
        print("Stored in AiiDA: "+repr(s))

inp_descr = widgets.Text(placeholder="Description (optional)")   
btn_store = widgets.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(widgets.HBox([btn_store, inp_descr]))